# Assignment 7

## 2. Beta and expected returns

a)

In [1]:
import wrds
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
db = wrds.Connection()
db.create_pgpass_file()

Loading library list...
Done
Enter your WRDS username [jonkq]:jonkq
Enter your password:········


In [3]:
# get rf
rf=db.raw_sql("select mcaldt, tmytm from crsp.tfz_mth_rf where kytreasnox = 2000001 and mcaldt>='1980-01-01' and mcaldt<='2019-12-31'")
rf.to_csv('rf.csv')
# get value-weighted CRSP market return
vwret = db.raw_sql("select date, vwretd from crsp.msi where date>='1980-01-01' and date<='2019-12-31'")
vwret.to_csv('vwret.csv')

In [4]:
rf = pd.read_csv('rf.csv', usecols=['tmytm']).squeeze()
#Compute the monthly simple rate
rf_m = np.exp(rf/12/100)-1

vwret = pd.read_csv('vwret.csv', usecols=['vwretd']).squeeze()

In [5]:
stocks = db.raw_sql("""
                select a.permno, a.date, a.ret, b.exchcd, b.shrcd, a.shrout, a.prc 

                from crsp.msf as a left join crsp.msenames as b 
                
                on a.permno = b.permno and b.namedt <= a.date and a.date<=b.nameendt
                
                where a.date between '1980-01-01' and '2019-12-31'
                and b.exchcd between 1 and 2
                and b.shrcd between 10 and 11""",date_cols=['date'])

stocks.to_csv('stocks.csv')

In [8]:
stocks_df = pd.pivot_table(data=stocks, values='ret', index='date', columns='permno')

In [36]:
s = stocks_df.apply(sum)

In [37]:
s = list(s.isna())

In [43]:
s_i = []
for i in range(len(s)):
    if not s[i]:
        s_i.append(i)

In [46]:
stocks_df = stocks_df.iloc[:,s_i]

In [50]:
vwret_e = vwret - rf_m

In [53]:
stocks_e = stocks_df - rf_m

In [55]:
stocks_df.iloc[:2,0]

date
1980-01-31    0.150127
1980-02-29   -0.013274
Name: 10145.0, dtype: float64

In [56]:
rf_m[:2]

0    0.009816
1    0.011412
Name: tmytm, dtype: float64

In [72]:
stocks_e.iloc[:2,0]

date
1980-01-31    0.140311
1980-02-29   -0.024686
Name: 10145.0, dtype: float64

In [70]:
rf_m.index = stocks_df.index
stocks_e = stocks_df.subtract(rf_m, axis = 0)